## Frequency Shift Chirp Modulation: La modulación LoRa

LoRa emplea **Chirps** como base de cada símbolo modulado. Un **chirp** es una señal senoidal que aumenta o decrementa su frecuencia de forma lineal.


Cada símbolo $s(nT_s)$ es un numero real que se genera a partir de un vector binario $w(nT_s)$ de longitud igual al **Spreading Factor** (SF), el cual puede tomar valores entre 7 y 12.


- **Vector de bits**: $w(nT_s) = [w_0(nT_s), \dots, w_{\mathrm{SF}-1}(nT_s)]$, donde cada $w_h(nT_s) \in \{0, 1\}$.  

- **Combinaciones posibles**: este vector de $\mathrm{SF}$ bits admite $2^{\mathrm{SF}}$ símbolos distintos.



Para convertir el vector binario $w(nT_s)$ a un valor real $s(nT_s)$, usamos la siguiente  ecuación propuesta en el paper de Vangelista:

$$
s(nT_s) \;=\; \sum_{h=0}^{\mathrm{SF}-1} w_h(nT_s)\,\cdot\,2^{h}
$$

donde:

* $w_h(nT_s)$ es el bit en la posición $h$ del vector de bits $w(nT_s)$.

* Cada bit se multiplica por $2^h$ (su “peso”) y se suman todos los términos.


Podemos ver entonces que el símbolo $s(nT_s)$ puede tomar cualquiera de los valores enteros:

$$
s(nT_s) \;\in\; \{\,0, 1, 2, \dots, 2^{\mathrm{SF}} -1\}
$$




In [12]:
import numpy as np

def generate_random_uniform_bits(sf, nb_samples, ref):
    nb_samples_adj = (nb_samples // sf) * sf    # Ajustamos nb_samples al múltiplo de sf más cercano por debajo
    z = np.random.uniform(size=nb_samples_adj)
    bits = (z > ref).astype(np.uint8)
    return bits

def encoder(bits, sf):
    n_blocks = bits.size // sf
    blocks = bits.reshape(n_blocks, sf)  # Restructuramos los bits en bloques de sf bits
    weights = 2 ** np.arange(sf)[::-1]   # Generamos un vector con los pesos de cada bit
    symbols = blocks.dot(weights)        # Producto punto entre bits y pesos
    return symbols

def decoder(symbols, sf):
    
    #symbols: array 1D de enteros (cada uno ∈ [0, 2**sf))
    #sf: número de bits por símbolo
    #Devuelve un array de forma (n_blocks, sf) con 0/1.
    
    symbols = np.asarray(symbols, dtype=np.uint64)
    # Creamos un vector de pesos [2^(sf-1), 2^(sf-2), ..., 1]
    weights = 2 ** np.arange(sf - 1, -1, -1, dtype=np.uint64)
    # Para cada símbolo comprobamos bit a bit si está activado
    bits = ((symbols[:, None] & weights[None, :]) > 0).astype(np.uint8)
    return bits

# Ejemplo de uso:
SF = 9                     # puede ser cualquiera entre 7 y 12
nb_samples = 10000
bits = generate_random_uniform_bits(SF, nb_samples, 0.5)

print(bits[:9])           # los primeros 9 bits

bits = decoder(encoder(bits, SF), SF)

print(bits[0])           # los primeros 9 bits



[1 0 1 1 0 1 0 0 1]
[1 0 1 1 0 1 0 0 1]
